# Timer

In [2]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 24, 10, 25

while True:
    dt = datetime.now()
    if dt.month >= month and dt.day >= day:
        if dt.hour > hour or (dt.hour == hour and dt.minute > minute):
            break
    sleep(60)

# Parameter optimization

In [3]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 50
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['indicator_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat = stat.sort_values(['indicator_rank', 'pct_right_forecast_avg'], ascending=False)
stat.head(20)

Number of combinations is 20


100%|██████████████████████████████████████████████████████████████████████| 20/20 [02:50<00:00,  8.54s/it]


,pattern,PriceChange_low_price_quantile,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,indicator_rank
0,PriceChange,1,100.00,0.00,0.00,0.00,0.00,0.00,0.00,50.00,50.00,50.00,50.00,0.00,50.00,50.00,0.00,0.00,0.00,0.00,50.00,50.00,50.00,50.00,50.00,100.00,0.83,-0.65,-2.10,-3.36,-3.71,-2.70,-3.09,-2.17,-1.92,-1.35,0.48,-0.76,-0.88,-1.58,-2.43,-2.61,-2.86,-3.19,-0.34,0.57,-0.12,0.41,2.54,7.02,2,31.250000,-77.500000
5,PriceChange,6,66.67,66.67,66.67,50.00,50.00,50.00,50.00,66.67,66.67,66.67,66.67,66.67,66.67,66.67,33.33,33.33,33.33,33.33,50.00,50.00,50.00,50.00,50.00,50.00,1.08,0.84,0.42,0.44,0.40,0.30,0.04,1.05,1.64,1.33,2.12,1.06,0.84,0.50,-1.03,-0.36,-0.67,-0.62,0.04,0.11,0.31,1.25,1.73,1.47,6,54.167500,-94.995000
2,PriceChange,3,75.00,50.00,50.00,50.00,50.00,25.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,25.00,25.00,25.00,25.00,50.00,50.00,50.00,50.00,50.00,50.00,1.59,0.36,-0.25,-0.64,-0.96,-0.30,0.04,0.23,0.48,0.33,0.31,-0.04,-0.04,-0.17,-1.03,-1.07,-1.33,-1.51,-0.36,-0.32,0.29,1.25,2.95,4.02,4,45.833333,-96.666667
1,PriceChange,2,66.67,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,0.00,0.00,0.00,0.00,33.33,33.33,33.33,33.33,33.33,33.33,1.09,-0.17,-0.80,-2.18,-2.31,-0.29,-0.50,-0.89,-0.89,-1.18,-1.48,-0.38,-0.59,-0.59,-1.18,-1.48,-1.48,-1.84,-1.18,-1.78,-0.89,-0.59,-0.89,-0.89,3,29.164167,-122.507500
3,PriceChange,4,60.00,60.00,60.00,40.00,40.00,40.00,40.00,60.00,60.00,60.00,60.00,60.00,60.00,60.00,20.00,20.00,20.00,20.00,40.00,40.00,40.00,40.00,40.00,40.00,1.09,0.80,0.30,-0.59,-0.38,-0.29,-0.50,0.76,1.43,1.02,2.10,0.30,0.50,0.25,-1.18,-0.67,-1.17,-1.18,-0.37,-0.91,-0.86,-0.59,-0.89,-0.89,5,45.000000,-125.000000
4,PriceChange,5,60.00,60.00,60.00,40.00,40.00,40.00,40.00,60.00,60.00,60.00,60.00,60.00,60.00,60.00,20.00,20.00,20.00,20.00,40.00,40.00,40.00,40.00,40.00,40.00,1.09,0.80,0.30,-0.59,-0.38,-0.29,-0.50,0.76,1.43,1.02,2.10,0.30,0.50,0.25,-1.18,-0.67,-1.17,-1.18,-0.37,-0.91,-0.86,-0.59,-0.89,-0.89,5,45.000000,-125.000000
6,PriceChange,7,77.78,66.67,55.56,55.56,44.44,44.44,44.44,44.44,44.44,55.56,44.44,55.56,44.44,44.44,44.44,44.44,44.44,44.44,55.56,55.56,44.44,44.44,44.44,55.56,0.69,0.80,0.30,0.42,-0.38,-0.88,-1.05,0.00,0.00,0.21,-0.42,0.30,-0.42,-0.21,-0.21,-0.42,0.00,0.00,0.35,0.11,-0.46,-0.51,-0.81,2.74,9,49.998750,-180.011250
9,PriceChange,10,81.82,54.55,54.55,54.55,36.36,36.36,36.36,36.36,36.36,54.55,54.55,63.64,54.55,54.55,54.55,54.55,45.45,45.45,54.55,54.55,54.55,54.55,54.55,63.64,0.96,0.80,0.30,0.19,-0.38,-0.61,-0.53,-0.35,-0.14,0.21,0.15,0.72,0.90,1.09,0.16,0.27,0.00,0.00,2.61,2.38,1.68,2.11,2.73,2.74,11,51.895833,-199.145833
10,PriceChange,11,83.33,58.33,58.33,58.33,41.67,33.33,41.67,41.67,41.67,58.33,50.00,58.33,50.00,50.00,50.00,50.00,41.67,41.67,50.00,50.00,58.33,58.33,58.33,66.67,0.99,0.84,0.42,0.30,-0.20,-0.55,-0.39,-0.17,-0.07,0.28,0.02,0.51,0.24,0.44,-0.03,-0.08,-0.19,-0.32,0.93,0.87,0.95,1.56,1.85,2.24,12,52.082917,-215.005000
8,PriceChange,9,80.00,50.00,50.00,50.00,30.00,30.00,30.00,30.00,30.00,50.00,50.00,60.00,50.00,50.00,50.00,50.00,40.00,40.00,50.00,

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [10]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)